# Cercador model òptim de LSTM i GRU de una capa

In [1]:
import numpy as np
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
import plotly.graph_objects as go
import math
import seaborn as sns
from sklearn.metrics import mean_squared_error
from time import time

np.random.seed(2)
tf.random.set_seed(2)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras import backend

C:\Users\dj_kr\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Càrrega de les dades

In [2]:
df = pd.read_csv('data/SentDATA.csv')

df.reset_index(inplace=True)
df['Time'] = pd.to_datetime(df['Time'])
df = df.set_index('Time')

## Transformació de dades

In [6]:
columns = ['PM1','PM25','PM10','PM1ATM','PM25ATM','PM10ATM']

df1 = df.copy();

df1 = df1.rename(columns={"PM 1":"PM1","PM 2.5":"PM25","PM 10":"PM10","PM 1 ATM":"PM1ATM","PM 2.5 ATM":"PM25ATM","PM 10 ATM":"PM10ATM"})

df1['PM1'] = df['PM 1'].astype(np.float32)
df1['PM25'] = df['PM 2.5'].astype(np.float32)
df1['PM10'] = df['PM 10'].astype(np.float32)
df1['PM1ATM'] = df['PM 1 ATM'].astype(np.float32)
df1['PM25ATM'] = df['PM 2.5 ATM'].astype(np.float32)
df1['PM10ATM'] = df['PM 10 ATM'].astype(np.float32)

In [11]:
df2 = df1.copy()

## Crear dades d'entrenament i de test

In [12]:
train_size = int(len(df2) * 0.8)
test_size = len(df2) - train_size
train, test = df2.iloc[0:train_size], df2.iloc[train_size:len(df2)]
train.shape, test.shape

((3991, 7), (998, 7))

## Normalitzar les dades d'entrenament

In [13]:
#Standardize the data
for col in columns:
    scaler = StandardScaler()
    train[col] = scaler.fit_transform(train[[col]])


<ipython-input-13-ad7c79e4e223>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-13-ad7c79e4e223>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-13-ad7c79e4e223>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-13-ad7c79e

## Crear finestra de temps PM 2.5

In [14]:
TIME_STEPS=144 #6 registres hora x 24h x 3 --> equival a una finestra d'un dia

def create_sequences(X, y, time_steps=TIME_STEPS):
    Xs, ys = [], []
    for i in range(len(X)-time_steps):
        Xs.append(X.iloc[i:(i+time_steps)].values)
        ys.append(y.iloc[i+time_steps])
    
    return np.array(Xs), np.array(ys)

X_train1h, y_train1h = create_sequences(train[[columns[1]]], train[columns[1]], 6) #1 hour

X_train3h, y_train3h = create_sequences(train[[columns[1]]], train[columns[1]], 18) #3 hours

X_train6h, y_train6h = create_sequences(train[[columns[1]]], train[columns[1]], 36) #6 hours

X_train12h, y_train12h = create_sequences(train[[columns[1]]], train[columns[1]], 72) #12 hours

X_train1d, y_train1d = create_sequences(train[[columns[1]]], train[columns[1]], 144) #1 day

X_train3d, y_train3d = create_sequences(train[[columns[1]]], train[columns[1]], 432) #3 days

X_train7d, y_train7d = create_sequences(train[[columns[1]]], train[columns[1]], 1008) #7 days
#X_test, y_test = create_sequences(test[[columns[1]]], test[columns[1]])

print(f'X_train1h shape: {X_train1d.shape}')
print(f'X_train3d shape: {X_train3h.shape}')
print(f'X_train6h shape: {X_train6h.shape}')
print(f'X_train12h shape: {X_train12h.shape}')
print(f'X_train1d shape: {X_train1d.shape}')
print(f'X_train3d shape: {X_train3d.shape}')
print(f'X_train7d shape: {X_train7d.shape}')

X_train1h shape: (3847, 144, 1)
X_train3d shape: (3973, 18, 1)
X_train6h shape: (3955, 36, 1)
X_train12h shape: (3919, 72, 1)
X_train1d shape: (3847, 144, 1)
X_train3d shape: (3559, 432, 1)
X_train7d shape: (2983, 1008, 1)


In [15]:
def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()

    print(model_name + ':')
    print('Mean Absolute Error: {:.4f}'.format(mae))
    print('Root Mean Square Error: {:.4f}'.format(rmse))
    print('Mean Square Error: {:.4f}'.format(mse))
    print('')
    return mae,rmse,mse

## Cerca dels models òptims

In [17]:
def _model(cmodel,units,activationDense,dropout1,optimizer):
    model = Sequential()
    model.add(cmodel(units = units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(rate=dropout1))
    model.add(TimeDistributed(Dense(1,kernel_initializer='normal',activation=activationDense)))
    model.compile(optimizer=optimizer, loss='mae')
    model.summary()
    
    return model

models = [LSTM,GRU]
nmodels = ["LSTM","GRU"]
sequences = ["1h","3h","6h","12h","1d", "3d", "7d"]
X_trains = [X_train1h,X_train3h, X_train6h, X_train12h,X_train1d,X_train3d, X_train7d]
y_trains= [y_train1h,y_train3h, y_train6h,y_train12h,y_train1d,y_train3d, y_train7d]
activations = ['relu']
activationsDense = ['tanh','sigmoid']
optimizers = ['adam','adadelta','adamax']
list_validationSplit = [0.1,0.2]
list_dropout1 =  np.random.uniform(0.1,0.8,5)   
list_units = np.random.randint(6,high=100, size=5) 
list_epochs = np.random.randint(5,high=100, size=5)   
list_batchsize = np.random.randint(6,high=64, size=5)     

list_results = pd.DataFrame()

for cmodel,nmodel in zip(models,nmodels):
    for X_train, y_train,sequence in zip(X_trains,y_trains,sequences):
        for optimizer in optimizers:
            #for activation in activations:
                for activationDense in activationsDense:
                    for units,epochs,batchsize,dropout1,validationsplit in zip(list_units,list_epochs,list_batchsize,list_dropout1,list_validationSplit): 
                        start = time()
                        print("###########################\n")
                        print("MODEL: ", nmodel)
                        print('sequence: ',sequence)
                        print('units: ',units)
                        print('dropout1: ',dropout1)
                        print('optimizer:',optimizer)
                        #print('activation:',activation)
                        print('activationDense:',activationDense)
                        print('epochs:',epochs)
                        print('batchsize:',batchsize)
                        print('validation_split:',validationsplit)
                    
                        model = _model(cmodel,units,activationDense,dropout1,optimizer)
                        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batchsize, validation_split=validationsplit,callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min')], shuffle=False)
                        end = time()
                        totalTime = end-start
                        print ('Execution time: ',totalTime)

                        X_train_pred = model.predict(X_train, verbose=0)
                        
                        mae,rmse,mse = evaluate_prediction(X_train_pred, X_train,nmodel)
                        
                        print('Train RMSE: %.3f' % rmse);
                        print('Train MSE: %.3f' % mse);
                        print('Train MAE: %.3f' % mae);
                        
                        result = pd.DataFrame({
                                               #'activation':[activation],
                                               'model':[nmodel],
                                               'sequence':[sequence],
                                               'activationDense':[activationDense],
                                               'optimizer':[optimizer],
                                               'dropout1':[dropout1],
                                               'units':[units],
                                               'epochs':[epochs],
                                               'batchsize':[batchsize],
                                               'validation_split':[validationsplit],
                                               
                                               'RMSE':[rmse],
                                               'MSE':[mse],
                                               'MAE':[mae],                            
                                               'Time':[totalTime]})
                        list_results = list_results.append(result) 

###########################

MODEL:  LSTM
sequence:  1h
units:  40
dropout1:  0.40519643149940265
optimizer: adam
activationDense: tanh
epochs: 68
batchsize: 45
validation_split: 0.1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 6, 40)             6720      
_________________________________________________________________
dropout (Dropout)            (None, 6, 40)             0         
_________________________________________________________________
time_distributed (TimeDistri (None, 6, 1)              41        
Total params: 6,761
Trainable params: 6,761
Non-trainable params: 0
_________________________________________________________________
Epoch 1/68
80/80 [==============================] - 1s 11ms/step - loss: 0.5607 - val_loss: 0.3392
Epoch 2/68
80/80 [==============================] - 0s 4ms/step - loss: 0.3897 - val_loss: 0.3060
Epoc

355/355 [==============================] - 2s 6ms/step - loss: 0.3993 - val_loss: 0.2529
Epoch 2/36
355/355 [==============================] - 1s 4ms/step - loss: 0.2935 - val_loss: 0.2201
Epoch 3/36
355/355 [==============================] - 1s 4ms/step - loss: 0.2740 - val_loss: 0.2121
Epoch 4/36
355/355 [==============================] - 1s 4ms/step - loss: 0.2690 - val_loss: 0.2104
Epoch 5/36
355/355 [==============================] - 1s 4ms/step - loss: 0.2678 - val_loss: 0.2091
Epoch 6/36
355/355 [==============================] - 1s 4ms/step - loss: 0.2673 - val_loss: 0.2083
Epoch 7/36
355/355 [==============================] - 1s 4ms/step - loss: 0.2670 - val_loss: 0.2075
Epoch 8/36
355/355 [==============================] - 1s 4ms/step - loss: 0.2667 - val_loss: 0.2064
Epoch 9/36
355/355 [==============================] - 1s 4ms/step - loss: 0.2662 - val_loss: 0.2054
Epoch 10/36
355/355 [==============================] - 1s 4ms/step - loss: 0.2659 - val_loss: 0.2051
Epoch 11/3

80/80 [==============================] - 0s 4ms/step - loss: 0.4933 - val_loss: 0.7928
Epoch 37/68
80/80 [==============================] - 0s 4ms/step - loss: 0.4931 - val_loss: 0.7928
Epoch 38/68
80/80 [==============================] - 0s 4ms/step - loss: 0.4929 - val_loss: 0.7928
Epoch 39/68
80/80 [==============================] - 0s 4ms/step - loss: 0.4932 - val_loss: 0.7928
Epoch 40/68
80/80 [==============================] - 0s 4ms/step - loss: 0.4929 - val_loss: 0.7927
Epoch 41/68
80/80 [==============================] - 0s 4ms/step - loss: 0.4927 - val_loss: 0.7927
Epoch 42/68
80/80 [==============================] - 0s 4ms/step - loss: 0.4926 - val_loss: 0.7927
Epoch 43/68
80/80 [==============================] - 0s 4ms/step - loss: 0.4927 - val_loss: 0.7927
Epoch 44/68
80/80 [==============================] - 0s 4ms/step - loss: 0.4928 - val_loss: 0.7927
Epoch 45/68
80/80 [==============================] - 0s 4ms/step - loss: 0.4926 - val_loss: 0.7927
Epoch 46/68
80/80 [===

Epoch 4/68
80/80 [==============================] - 0s 4ms/step - loss: 0.6784 - val_loss: 0.7680
Epoch 5/68
80/80 [==============================] - 0s 4ms/step - loss: 0.6778 - val_loss: 0.7677
Epoch 6/68
80/80 [==============================] - 0s 4ms/step - loss: 0.6777 - val_loss: 0.7674
Epoch 7/68
80/80 [==============================] - 0s 4ms/step - loss: 0.6776 - val_loss: 0.7670
Epoch 8/68
80/80 [==============================] - 0s 4ms/step - loss: 0.6774 - val_loss: 0.7667
Epoch 9/68
80/80 [==============================] - 0s 4ms/step - loss: 0.6770 - val_loss: 0.7663
Epoch 10/68
80/80 [==============================] - 0s 4ms/step - loss: 0.6769 - val_loss: 0.7660
Epoch 11/68
80/80 [==============================] - 0s 4ms/step - loss: 0.6767 - val_loss: 0.7656
Epoch 12/68
80/80 [==============================] - 0s 4ms/step - loss: 0.6763 - val_loss: 0.7653
Epoch 13/68
80/80 [==============================] - 0s 4ms/step - loss: 0.6763 - val_loss: 0.7649
Epoch 14/68
80/8

Epoch 42/68
80/80 [==============================] - 0s 4ms/step - loss: 0.8856 - val_loss: 1.2943
Epoch 43/68
80/80 [==============================] - 0s 4ms/step - loss: 0.8855 - val_loss: 1.2942
Epoch 44/68
80/80 [==============================] - 0s 4ms/step - loss: 0.8854 - val_loss: 1.2941
Epoch 45/68
80/80 [==============================] - 0s 4ms/step - loss: 0.8853 - val_loss: 1.2940
Epoch 46/68
80/80 [==============================] - 0s 4ms/step - loss: 0.8853 - val_loss: 1.2939
Epoch 47/68
80/80 [==============================] - 0s 4ms/step - loss: 0.8852 - val_loss: 1.2938
Epoch 48/68
80/80 [==============================] - 0s 4ms/step - loss: 0.8852 - val_loss: 1.2937
Epoch 49/68
80/80 [==============================] - 0s 4ms/step - loss: 0.8851 - val_loss: 1.2936
Epoch 50/68
80/80 [==============================] - 0s 4ms/step - loss: 0.8851 - val_loss: 1.2935
Epoch 51/68
80/80 [==============================] - 0s 4ms/step - loss: 0.8850 - val_loss: 1.2934
Epoch 52/6

...

Model: "sequential_156"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_72 (GRU)                 (None, 1008, 40)          5160      
_________________________________________________________________
dropout_156 (Dropout)        (None, 1008, 40)          0         
_________________________________________________________________
time_distributed_156 (TimeDi (None, 1008, 1)           41        
Total params: 5,201
Trainable params: 5,201
Non-trainable params: 0
_________________________________________________________________
Epoch 1/68
60/60 [==============================] - 23s 382ms/step - loss: 0.6499 - val_loss: 0.6928
Epoch 2/68
60/60 [==============================] - 23s 384ms/step - loss: 0.5896 - val_loss: 0.5294
Epoch 3/68
60/60 [==============================] - 23s 390ms/step - loss: 0.5761 - val_loss: 0.4939
Epoch 4/68
60/60 [==============================] - 24s 404ms/step - loss: 0.

60/60 [==============================] - 24s 402ms/step - loss: 0.6733 - val_loss: 0.8215
Epoch 15/68
60/60 [==============================] - 24s 402ms/step - loss: 0.6732 - val_loss: 0.8214
Epoch 16/68
60/60 [==============================] - 24s 404ms/step - loss: 0.6731 - val_loss: 0.8213
Epoch 17/68
60/60 [==============================] - 24s 400ms/step - loss: 0.6731 - val_loss: 0.8213
Epoch 18/68
60/60 [==============================] - 24s 401ms/step - loss: 0.6730 - val_loss: 0.8212
Epoch 19/68
60/60 [==============================] - 24s 398ms/step - loss: 0.6730 - val_loss: 0.8212
Epoch 20/68
60/60 [==============================] - 24s 405ms/step - loss: 0.6730 - val_loss: 0.8211
Epoch 21/68
60/60 [==============================] - 24s 402ms/step - loss: 0.6730 - val_loss: 0.8211
Epoch 22/68
60/60 [==============================] - 24s 397ms/step - loss: 0.6729 - val_loss: 0.8211
Epoch 23/68
60/60 [==============================] - 24s 403ms/step - loss: 0.6729 - val_loss:

60/60 [==============================] - 25s 410ms/step - loss: 0.6656 - val_loss: 0.8146
Epoch 49/68
60/60 [==============================] - 24s 405ms/step - loss: 0.6654 - val_loss: 0.8143
Epoch 50/68
60/60 [==============================] - 24s 403ms/step - loss: 0.6653 - val_loss: 0.8140
Epoch 51/68
60/60 [==============================] - 24s 403ms/step - loss: 0.6651 - val_loss: 0.8137
Epoch 52/68
60/60 [==============================] - 24s 404ms/step - loss: 0.6650 - val_loss: 0.8134
Epoch 53/68
60/60 [==============================] - 25s 411ms/step - loss: 0.6648 - val_loss: 0.8131
Epoch 54/68
60/60 [==============================] - 24s 402ms/step - loss: 0.6647 - val_loss: 0.8128
Epoch 55/68
60/60 [==============================] - 25s 411ms/step - loss: 0.6645 - val_loss: 0.8125
Epoch 56/68
60/60 [==============================] - 24s 405ms/step - loss: 0.6643 - val_loss: 0.8122
Epoch 57/68
60/60 [==============================] - 25s 410ms/step - loss: 0.6642 - val_loss:

Epoch 2/68
60/60 [==============================] - 25s 419ms/step - loss: 0.9758 - val_loss: 1.3136
Epoch 3/68
60/60 [==============================] - 25s 413ms/step - loss: 0.9758 - val_loss: 1.3135
Epoch 4/68
60/60 [==============================] - 25s 419ms/step - loss: 0.9758 - val_loss: 1.3134
Epoch 5/68
60/60 [==============================] - 25s 417ms/step - loss: 0.9757 - val_loss: 1.3134
Epoch 6/68
60/60 [==============================] - 25s 422ms/step - loss: 0.9757 - val_loss: 1.3133
Epoch 7/68
60/60 [==============================] - 26s 441ms/step - loss: 0.9756 - val_loss: 1.3132
Epoch 8/68
60/60 [==============================] - 27s 449ms/step - loss: 0.9756 - val_loss: 1.3132
Epoch 9/68
60/60 [==============================] - 27s 444ms/step - loss: 0.9755 - val_loss: 1.3131
Epoch 10/68
60/60 [==============================] - 27s 444ms/step - loss: 0.9754 - val_loss: 1.3130
Epoch 11/68
60/60 [==============================] - 27s 443ms/step - loss: 0.9754 - val_l

Epoch 36/68
60/60 [==============================] - 24s 397ms/step - loss: 0.5507 - val_loss: 0.4180
Epoch 37/68
60/60 [==============================] - 24s 395ms/step - loss: 0.5504 - val_loss: 0.4188
Epoch 38/68
60/60 [==============================] - 24s 393ms/step - loss: 0.5500 - val_loss: 0.4198
Epoch 39/68
60/60 [==============================] - 24s 394ms/step - loss: 0.5497 - val_loss: 0.4208
Epoch 40/68
60/60 [==============================] - 24s 401ms/step - loss: 0.5493 - val_loss: 0.4221
Execution time:  968.2555317878723
GRU:
Mean Absolute Error: 0.6504
Root Mean Square Error: 1.1419
Mean Square Error: 1.3039

Train RMSE: 1.142
Train MSE: 1.304
Train MAE: 0.650
###########################

MODEL:  GRU
sequence:  7d
units:  55
dropout1:  0.11814836227952394
optimizer: adamax
activationDense: tanh
epochs: 36
batchsize: 9
validation_split: 0.2
Model: "sequential_165"
_________________________________________________________________
Layer (type)                 Output Sha

In [18]:
list_results.to_csv("resultats-cerca-optim-lstm-una-capa.csv")

In [19]:
list_results.head(20)

,model,sequence,activationDense,optimizer,dropout1,units,epochs,batchsize,validation_split,RMSE,MSE,MAE,Time
0,LSTM,1h,tanh,adam,0.405196,40,68,45,0.1,0.581774,0.338461,0.159277,27.412817
0,LSTM,1h,tanh,adam,0.118148,55,36,9,0.2,0.581916,0.338626,0.151122,51.123055
0,LSTM,1h,sigmoid,adam,0.405196,40,68,45,0.1,0.752527,0.566297,0.483684,28.099232
0,LSTM,1h,sigmoid,adam,0.118148,55,36,9,0.2,0.758053,0.574645,0.483228,35.801286
0,LSTM,1h,tanh,adadelta,0.405196,40,68,45,0.1,0.958890,0.919471,0.669135,27.464905
0,LSTM,1h,tanh,adadelta,0.118148,55,36,9,0.2,0.932710,0.869947,0.639812,51.920173
0,LSTM,1h,sigmoid,adadelta,0.405196,40,68,45,0.1,1.118657,1.251392,0.923693,29.166857
0,LSTM,1h,sigmoid,adadelta,0.118148,55,36,9,0.2,1.103161,1.216963,0.908986,52.628776
0,LSTM,1h,tanh,adamax,0.405196,40,68,45,0.1,0.584879,0.342083,0.161601,27.472851
0,LSTM,1h,tanh,adamax,0.118148,55,36,9,0.2,0.582802,0.339658,0.153627,51.269109


In [22]:
list_results.sort_values(by=['RMSE', 'sequence'])

,model,sequence,activationDense,optimizer,dropout1,units,epochs,batchsize,validation_split,RMSE,MSE,MAE,Time
0,GRU,1h,tanh,adam,0.118148,55,36,9,0.2,0.581650,0.338317,0.154136,58.817262
0,LSTM,1h,tanh,adam,0.405196,40,68,45,0.1,0.581774,0.338461,0.159277,27.412817
0,GRU,1h,tanh,adamax,0.118148,55,36,9,0.2,0.581867,0.338569,0.154247,58.510861
0,LSTM,1h,tanh,adam,0.118148,55,36,9,0.2,0.581916,0.338626,0.151122,51.123055
0,GRU,1h,tanh,adam,0.405196,40,68,45,0.1,0.582345,0.339125,0.160290,30.322881
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,LSTM,7d,tanh,adam,0.405196,40,68,45,0.1,1.146864,1.315297,0.643848,1682.198702
0,LSTM,7d,tanh,adamax,0.405196,40,68,45,0.1,1.154867,1.333718,0.664537,1262.185469
0,LSTM,7d,tanh,adam,0.118148,55,36,9,0.2,1.158497,1.342116,0.648923,2014.745057
0,LSTM,7d,tanh,adamax,0.118148,55,36,9,0.2,1.160135,1.345913,0.653456,1424.883327
